### JAI SHREE RAM

Stage 2: Predicting Length of Stay

In [202]:
import pandas as pd
import numpy as np

In [203]:
#import admisson data
df = pd.read_csv('admission_data.csv')

In [204]:
df

,PID,AGE,GENDER,SMOKING,ALCOHOL,HTN,CAD,PRIOR CMP,HB,TLC,...,VT,PSVT,CONGENITAL,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION,TARGET
0,506,50,M,0,0,1,1,0,13.1,10.5,...,0,0,0,0,0,0,0,0,0,4
1,798,71,M,0,0,0,1,0,12.9,8.5,...,0,0,0,0,0,0,0,0,0,2
2,798,71,M,0,0,0,1,0,12.9,8.5,...,0,0,0,0,0,0,0,0,0,2
3,798,72,M,0,0,0,1,0,11.4,6.6,...,0,0,0,0,0,0,0,0,0,6
4,989,71,M,0,0,1,1,0,10.7,14.1,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15752,4888286,68,M,0,0,0,1,0,12.2,6.7,...,0,0,0,0,0,0,0,0,0,5
15753,4888926,65,F,0,0,1,1,0,11.3,11.3,...,0,0,0,0,0,0,0,0,0,2
15754,5711587,60,F,0,0,0,0,1,10.1,9.8,...,0,0,0,0,0,0,0,0,0,11
15755,6408503,68,F,0,0,0,1,0,9.8,17.3,...,0,0,0,0,0,0,1,0,0,11


In [205]:
#find correlation between all the variables
corr_matrix = df.corr()
#remove correlated variables above 0.80
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
# Find features with correlation greater than 0.80
to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]
# Drop features 
df.drop(to_drop, axis=1, inplace=True)

C:\Users\purja\AppData\Local\Temp\ipykernel_8852\997048037.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df.corr()


In [206]:
#list unique values in chest infection
df['CHEST INFECTION'].unique()
#drop rows with '\\' in chest infection
df = df[df['CHEST INFECTION'] != '\\']

In [207]:
df.dropna(inplace=True)
df = df[df['PID'] != 'NILL']
df = df[(df != 'EMPTY').all(1)]
df['GENDER'] = np.where(df['GENDER'] == 'M', 1, 0)
#convert PID to int
df['PID'] = df['PID'].astype(int)
#convert HB and TLC to float 
df['HB'] = df['HB'].astype(float)
df['TLC'] = df['TLC'].astype(float)
#convert Platelets, glucose and chest infection to int
# df['PLATELETS'] = df['PLATELETS'].astype(float)
# df['GLUCOSE'] = df['GLUCOSE'].astype(float)
df['CHEST INFECTION'] = df['CHEST INFECTION'].astype(int)


C:\Users\purja\AppData\Local\Temp\ipykernel_8852\1421068524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [208]:
df.dtypes

PID                            int32
AGE                            int64
GENDER                         int32
SMOKING                        int64
ALCOHOL                        int64
HTN                            int64
CAD                            int64
PRIOR CMP                      int64
HB                           float64
TLC                          float64
PLATELETS                     object
GLUCOSE                       object
RAISED CARDIAC ENZYMES         int64
SEVERE ANAEMIA                 int64
ANAEMIA                        int64
STABLE ANGINA                  int64
ACS                            int64
STEMI                          int64
ATYPICAL CHEST PAIN            int64
HEART FAILURE                  int64
VALVULAR                       int64
CHB                            int64
SSS                            int64
AF                             int64
VT                             int64
PSVT                           int64
CONGENITAL                     int64
N

In [209]:
#list unique values and counts of PID in descending order
df['PID'].value_counts()
#find range of PID
print(df['PID'].max())
df['PID'].min()

5711587


506

In [210]:
#check if input PID already exists in DF return TARGET variable
def check_pid(pid):
    if pid in df['PID'].values:
        #return mean of all values for PID
        return df[df['PID'] == pid]['TARGET'].mean().round()
    else:
        return 0

In [211]:
#regression model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#split data into train and test and drop Target and PID variable from X
X = df.drop(['TARGET'], axis=1)
y = df['TARGET']

In [212]:
#fit the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
model = LinearRegression().fit(X_train, y_train)
#predict the model and convert to int 
# y_pred = model.predict(X_test)

In [213]:
#list data types
df.dtypes

PID                            int32
AGE                            int64
GENDER                         int32
SMOKING                        int64
ALCOHOL                        int64
HTN                            int64
CAD                            int64
PRIOR CMP                      int64
HB                           float64
TLC                          float64
PLATELETS                     object
GLUCOSE                       object
RAISED CARDIAC ENZYMES         int64
SEVERE ANAEMIA                 int64
ANAEMIA                        int64
STABLE ANGINA                  int64
ACS                            int64
STEMI                          int64
ATYPICAL CHEST PAIN            int64
HEART FAILURE                  int64
VALVULAR                       int64
CHB                            int64
SSS                            int64
AF                             int64
VT                             int64
PSVT                           int64
CONGENITAL                     int64
N

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend
from tensorflow.keras import models, regularizers, layers, optimizers, losses, metrics
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Training the network
modell = models.Sequential()
cols = X_train.shape[1]
input_shape = (cols,)
modell.add(layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=input_shape))
# model.add(layers.BatchNormalization())
modell.add(layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
modell.add(layers.Dropout(0.2))
modell.add(layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
modell.add(layers.Dropout(0.2))
modell.add(layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
modell.add(layers.Dropout(0.2))
modell.add(layers.Dense(1, activation='relu'))
print(modell.summary())

# FIT / TRAIN model on training daa
NumEpochs = 20
BatchSize = 8

X_train = np.asarray(X_train).astype('float32')
# Compile the model and fit it on the training data
modell.compile(optimizer=optimizers.Adam(lr=1e-4), loss='mean_absolute_error', metrics=['mae'])
history = modell.fit(X_train, y_train, epochs=NumEpochs, batch_size=BatchSize, validation_data=(X_test, y_test))

# Predicting on the test dataset
results = modell.evaluate(X_test, y_test)
print("_"*100)
print("Test Loss and MAE")
print("results ", results)

In [214]:
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test).astype(int).round()
y_pred.shape
#call check_pid function to check if PID already exists in DF for every row in y_pred
for i in range(len(X_test)):
    y_pred[i] = ((y_pred[i] + check_pid(X_test.iloc[i]['PID']))/2).round()

print(y_pred)
score = r2_score(y_test, y_pred)
print(score)

[8 9 6 ... 7 8 6]
0.6847353571088057


In [ ]:
# #predict the model
# test = np.array([989,71,1,0,0,1,1,0,10.7,14.1,227,253,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
# test = test.reshape(1, -1)
# pid = 989
# y_pred = model.predict(test).astype(int).round()
# ((y_pred[0] + check_pid(pid))/2).round()

In [233]:
#append X_test and y_test to df
df_test = X_test
df_test['LOS'] = y_pred

In [234]:
df_test

,PID,AGE,GENDER,SMOKING,ALCOHOL,HTN,CAD,PRIOR CMP,HB,TLC,...,VT,PSVT,CONGENITAL,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION,LOS
12775,602495,82,0,0,0,1,1,0,11.8,10.1,...,0,0,0,0,0,0,0,0,0,8
14935,670926,65,1,0,0,1,1,0,8.7,15.6,...,0,0,0,0,0,0,0,0,0,9
4006,280038,53,0,0,0,1,1,0,13.9,12.5,...,0,0,0,0,0,0,0,0,0,6
15095,675773,62,1,0,0,1,1,0,9.4,16.7,...,0,0,0,0,0,0,0,0,0,8
5078,315883,60,1,0,0,0,0,1,13.8,14.4,...,0,0,0,0,0,0,0,1,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6029,351289,66,0,0,0,1,0,1,12.7,9.2,...,0,0,0,0,0,0,0,0,0,6
7660,399243,45,1,0,1,0,0,1,13.8,7.0,...,0,0,0,0,0,0,0,0,0,6
9509,465403,42,1,1,1,1,1,0,16.9,12.6,...,0,0,0,0,0,0,0,0,0,7
8762,438244,60,1,0,0,1,1,0,12.6,8.0,...,0,0,0,0,0,0,0,0,0,8


In [235]:
#randomly select 29 rows from df_test
df_test = df_test.sample(n=43)

In [236]:
df_test

,PID,AGE,GENDER,SMOKING,ALCOHOL,HTN,CAD,PRIOR CMP,HB,TLC,...,VT,PSVT,CONGENITAL,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION,LOS
13545,629532,57,1,0,0,1,0,0,11.2,16.9,...,1,0,0,0,0,0,0,0,0,6
11753,560725,49,1,0,0,0,1,0,14.9,9.1,...,0,0,0,0,0,0,0,0,0,7
15493,691331,67,0,0,0,0,0,1,11.0,10.2,...,0,0,0,0,0,0,0,0,0,7
7835,405568,63,1,0,0,0,1,0,14.9,11.9,...,0,0,0,0,0,0,0,0,0,4
9030,448255,35,1,0,0,0,1,0,11.1,9.1,...,0,0,0,0,0,0,0,0,0,21
3973,279527,68,1,0,0,1,1,1,16.0,8.1,...,0,0,0,0,0,0,0,0,0,6
6239,357648,50,0,0,0,0,0,0,12.0,16.0,...,0,0,0,0,0,0,0,0,0,8
10324,497388,62,1,0,0,0,0,0,12.3,11.1,...,0,0,0,0,0,0,0,0,0,5
6876,379146,61,1,1,1,1,1,0,11.1,10.2,...,0,0,0,0,0,0,0,0,0,6
12928,607502,72,1,0,0,0,1,0,11.1,3.5,...,0,0,0,0,0,0,0,0,0,4


In [237]:
#read isAdmitted data
df_admitted = pd.read_csv('isAdmitted.csv')
df_admitted.drop(['Unnamed: 0'], axis=1, inplace=True)
df_admitted

,Predicted,Urgency Score
0,1,0.986587
1,1,0.970943
2,1,0.951988
3,1,0.943976
4,1,0.939065
5,1,0.938578
6,1,0.934265
7,1,0.927789
8,1,0.920569
9,1,0.914812


In [238]:
PID = df_test['PID']
PID = PID.to_numpy()

#append PID to df_admitted
df_admitted['PID'] = PID

In [239]:
df_admitted

,Predicted,Urgency Score,PID
0,1,0.986587,629532
1,1,0.970943,560725
2,1,0.951988,691331
3,1,0.943976,405568
4,1,0.939065,448255
5,1,0.938578,279527
6,1,0.934265,357648
7,1,0.927789,497388
8,1,0.920569,379146
9,1,0.914812,607502


In [240]:
#merge df_test and df_admitted
df_test = pd.merge(df_test, df_admitted, on='PID', how='right')

In [241]:
df_test['Length of Stay'] = df_test['LOS']


In [242]:
df_test.drop(['LOS'], axis=1, inplace=True)

In [244]:
#urgency score * 100
df_test['Urgency Score'] = df_test['Urgency Score'] * 100
df_test

,PID,AGE,GENDER,SMOKING,ALCOHOL,HTN,CAD,PRIOR CMP,HB,TLC,...,CONGENITAL,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION,Predicted,Urgency Score,Length of Stay
0,629532,57,1,0,0,1,0,0,11.2,16.9,...,0,0,0,0,0,0,0,1,98.658681,6
1,560725,49,1,0,0,0,1,0,14.9,9.1,...,0,0,0,0,0,0,0,1,97.094252,7
2,691331,67,0,0,0,0,0,1,11.0,10.2,...,0,0,0,0,0,0,0,1,95.198834,7
3,405568,63,1,0,0,0,1,0,14.9,11.9,...,0,0,0,0,0,0,0,1,94.397562,4
4,448255,35,1,0,0,0,1,0,11.1,9.1,...,0,0,0,0,0,0,0,1,93.906534,21
5,279527,68,1,0,0,1,1,1,16.0,8.1,...,0,0,0,0,0,0,0,1,93.857752,6
6,357648,50,0,0,0,0,0,0,12.0,16.0,...,0,0,0,0,0,0,0,1,93.426457,8
7,497388,62,1,0,0,0,0,0,12.3,11.1,...,0,0,0,0,0,0,0,1,92.778947,5
8,379146,61,1,1,1,1,1,0,11.1,10.2,...,0,0,0,0,0,0,0,1,92.056868,6
9,607502,72,1,0,0,0,1,0,11.1,3.5,...,0,0,0,0,0,0,0,1,91.481225,4


Stage 3: Estimating number of beds

In [370]:
def sort_queue(df):
    df = df.sort_values(by=['Urgency Score'], ascending=False)
    return df

In [368]:
totalbeds = 450
filledbeds = 420

#queue of urgency score and LOS only 20 rows from df_test
new = 0
new+=20
queue = df_test[['Urgency Score', 'Length of Stay']].head(new).to_numpy()

#add another column with value 0
queue = np.insert(queue, 2, 0, axis=1)
beds = np.zeros(450)
#make 400 in beds 1
for i in range(filledbeds):
    beds[i] = 1
i+=1
days = 1
while True: #real time
    print("Day No.: ",days)
    
    #decrement LOS by 1 for every patient with a bed
    for j in range(len(queue)):
        if queue[j,2] == 1:
            queue[j,1] -= 1
            #keep max of 0 or queue[j,1]
            if queue[j,1] < 0:
                queue[j,1] = 0
            #if LOS is 0, release the bed
            if queue[j,1] == 0:
                print("Released bed is: ", j+1)
                # beds[filledbeds+j] = 0
                i-=1
                queue[j,2] = -1 #discharged

    print("No. of beds available: ",450-i)
    #check if any patient does not have a bed and find the patient with highest urgency score
    while True:
        if i < 450:
            for j in range(len(queue)):
                if queue[j,2] == 0:
                    maxi = j
                    break
            if queue[maxi,2] == 0:
                # beds[i] = 1
                queue[maxi,2] = 1
                print("Bed filled for patient: ", maxi+1, " with urgency score: ", queue[maxi,0])
                i+=1
                #end loop if all patients are discharged
                if np.count_nonzero(queue[:,2] == 1) + np.count_nonzero(queue[:,2] == -1) == len(queue):
                    break
        else:
            break
    # print(queue)
    
    #print a new line
    print()
    days+=1
    #select next 10 rows from df_test
    if (new > len(df_test)):
        queue_new = df_test[['Urgency Score', 'Length of Stay']].iloc[new:].to_numpy()
        queue_new = np.insert(queue_new, 2, 0, axis=1)
        queue = np.concatenate((queue, queue_new), axis=0)
        
    else:
        queue_new = df_test[['Urgency Score', 'Length of Stay']].iloc[new:new+10].to_numpy()
        queue_new = np.insert(queue_new, 2, 0, axis=1)
        queue = np.concatenate((queue, queue_new), axis=0)
        new+=10
    
    #check if queue[:,2] is all 1 or -1
    if np.count_nonzero(queue[:,2] == 1) + np.count_nonzero(queue[:,2] == -1) == len(queue):
        break

print(days-1)


Day No.:  1
No. of beds available:  30
Bed filled for patient:  1  with urgency score:  98.65868056561786
Bed filled for patient:  2  with urgency score:  97.09425175571175
Bed filled for patient:  3  with urgency score:  95.19883355558828
Bed filled for patient:  4  with urgency score:  94.39756224096548
Bed filled for patient:  5  with urgency score:  93.9065339860748
Bed filled for patient:  6  with urgency score:  93.85775213254229
Bed filled for patient:  7  with urgency score:  93.42645706698916
Bed filled for patient:  8  with urgency score:  92.77894671221819
Bed filled for patient:  9  with urgency score:  92.05686810506802
Bed filled for patient:  10  with urgency score:  91.48122522737741
Bed filled for patient:  11  with urgency score:  89.63668180262522
Bed filled for patient:  12  with urgency score:  89.37653136324097
Bed filled for patient:  13  with urgency score:  88.85482306852532
Bed filled for patient:  14  with urgency score:  86.05853744652475
Bed filled for pati